 #  corr相关系数筛选特征

In [2]:
import pandas as pd
train_df=pd.read_csv('./happiness_train_complete.csv',index_col=0,encoding='gb2312')
test_df=pd.read_csv('./happiness_test_complete.csv',index_col=0,encoding='gb2312')
import  seaborn as sns#绘图库
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV 
#填充缺失值
from sklearn.preprocessing import Imputer
from sklearn.metrics import fbeta_score, make_scorer
from xgboost.sklearn import XGBRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [3]:
y=train_df.happiness
y[y<0]=0.0001
y.shape

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(8000,)

In [4]:
train_df.drop(['happiness'],axis=1,inplace=True)
all_data=pd.concat([train_df,test_df])#拼接两个数据集，便于统一操作

In [5]:
cate_columns=[
             'province','city','county','gender','nationality','edu','edu_status'
             ,'political','health','hukou','hukou_loc','work_exper'
             ,'work_status','family_m','son','daughter'
              ,'minor_child','marital','s_edu','s_political','s_hukou','s_work_exper'
             ,'f_edu','f_work_14','m_edu','m_political','m_work_14'
            ]
del_columns=['survey_time','edu_other','edu_yr','join_party','property_other',
             'work_yr','work_type','work_manage','invest_other','s_work_type'
             ,'f_birth','m_birth','inc_exp','marital_1st','marital_now','s_work_status']
num_columns=[cat for cat in all_data if cat not in cate_columns and cat not in del_columns]


In [6]:
all_data_dummies=all_data.drop(del_columns,axis=1)#删除感觉没用的列

all_data_dummies = pd.get_dummies(data=all_data_dummies,columns=cate_columns)#对部分特征进行one hot编码

In [7]:
#归一化
s = (all_data_dummies[num_columns] - all_data_dummies[num_columns].min())/(all_data_dummies[num_columns].max() - all_data_dummies[num_columns].min())
all_data_dummies[s.columns] = s


In [8]:
im = Imputer(strategy="most_frequent",axis=1)#众数填充
filled_nan_data = im.fit_transform(all_data_dummies)
filled_nan_data=pd.DataFrame(filled_nan_data,index=range(1,len(all_data)+1))
filled_nan_data.columns=all_data_dummies.columns

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
X =filled_nan_data.loc[train_df.index]#重新分开数据，
X_test=filled_nan_data.loc[test_df.index]#X_val 就是要预测的x
X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=0.9,random_state=10)

X_train.shape

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(7200, 564)

In [23]:
from sklearn.feature_selection import VarianceThreshold #导入python的相关模块
sel=VarianceThreshold(threshold=(0.01))#表示剔除特征的方差大于阈值的特征Removing features with low variance
col_selected=sel.fit_transform(X_train)#返回的结果为选择的特征矩阵

col_selected_val=sel.transform(X_val)
col_selected_val.shape

(800, 229)

In [9]:
#使用Lasso筛选特征

from sklearn.svm import LinearSVR
from sklearn.feature_selection import SelectFromModel


lsvc = LinearSVR(C=0.01).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)
X_train= model.transform(X_train)
X_val=model.transform(X_val)

In [10]:
X_train.shape,X_val.shape

((7200, 172), (800, 172))

In [25]:
#调参
from sklearn.linear_model import BayesianRidge
from sklearn.svm import  SVR
from sklearn.neighbors import KNeighborsRegressor
cv_params = {
#             'n_neighbors':[3,4,5,6,7]
#             'learning_rate': [0.5,0.4,0.3,0.2,0.1,0.075,0.05,0.04,0.03],
#             'subsample':[ 0.1 * i for i in range(6,9)],
#             'colsample_bytree':[ 0.1 * i for i in range(6,9)],
#             'gamma':[ 0.1 * i for i in range(0,5)],
#             'max_depth': list(range(4,9)), 
#             'min_child_weight': list((1,3,6))
}
# cv_params = {'n_estimators': [100,150]}
other_params = { 'max_depth': 3, 'min_child_weight': 1, 'seed': 0,'random_state':42,'learning_rate':0.105,
                'subsample':0.89, 'colsample_bytree': 0.7, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1,'n_jobs':-1}

# model = KNeighborsRegressor()
model = XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='neg_mean_squared_error', cv=5, verbose=3)
optimized_GBM.fit(X=col_selected,y= (y_train))
print(optimized_GBM.best_estimator_)
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
y_pred=optimized_GBM.best_estimator_.predict(col_selected_val)
print('泛化性能均方误差:',np.mean((y_val-(y_pred))**2))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................... , score=-0.5758523651534384, total=   3.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV] ...................... , score=-0.5440701986115977, total=   3.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.5s remaining:    0.0s


[CV] ...................... , score=-0.5091057615212295, total=   3.0s
[CV]  ................................................................
[CV] ...................... , score=-0.4886340268651916, total=   3.0s
[CV]  ................................................................
[CV] ...................... , score=-0.5486700381407981, total=   3.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   16.0s finished


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.105,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None, objective='reg:linear',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=0, silent=True, subsample=0.89)
参数的最佳取值：{}
最佳模型得分:-0.5332664780584511
泛化性能均方误差: 0.5702443141216311


In [12]:
#模型的融合
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.svm import  SVR
#Stacking

clfs = [
          BayesianRidge()
          ,SVR()
        ,KNeighborsRegressor()
]
#训练过程
dataset_stack_train = np.zeros((X_train.shape[0],len(clfs)))
dataset_stack_test = np.zeros((X_val.shape[0],len(clfs)))
for j,clf in enumerate(clfs):
    clf.fit(X_train,y_train)
    y_submission = clf.predict(X_val)
    y_train_tmp = clf.predict(X_train)
    dataset_stack_train[:,j] = y_train_tmp
    dataset_stack_test[:,j] = y_submission
print("开始Stacking....")

clf = Ridge()
# clf=XGBRegressor(max_depth=3,learning_rate=0.105,n_estimators=100,subsample=0.89,silent=False)
clf.fit(dataset_stack_train,y_train)
y_submission = clf.predict(dataset_stack_test)
print(np.mean((y_val-y_submission)**2))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


开始Stacking....
0.6691160401712317


In [13]:
y_submission

array([ 3.91097775,  4.38759991,  3.8120743 ,  3.67992451,  4.08574046,
        4.26468895,  3.8994642 ,  4.23179077,  3.66791729,  3.80613252,
        4.29502396,  4.20188269,  3.49610402,  3.4297816 ,  3.65543183,
        3.81336567,  3.04709085,  4.4646575 ,  4.09828984,  3.83064743,
        3.22089725,  3.73753126,  3.48514453,  4.4701798 ,  3.40026291,
        5.007425  ,  3.44863315,  3.78092235,  3.96324569,  3.31465746,
        3.40969795,  4.02118905,  3.52428821,  4.23620447,  3.83047111,
        3.77262241,  4.23237899,  3.99426322,  3.79947856,  4.53300147,
        3.31046271,  3.83244878,  4.11827161,  3.33619952,  4.38936828,
        4.01010314,  3.30790008,  3.63041191,  4.48192178,  3.74506258,
        4.28957322,  2.49601323,  3.29979083,  3.8974856 ,  3.48809577,
        3.70071687,  3.97685236,  2.75707397,  4.21911121,  4.66282895,
        2.67350387,  3.77337948,  2.88617063,  3.66825337,  3.3698808 ,
        3.42523404,  4.11295599,  3.76172863,  3.78501679,  4.29